In [1]:
import torch
from torch import nn

# Dropout

In [2]:
x = torch.randn(3,7)
drop = nn.Dropout(p=0.7) # p는 죽일 확률
# 각 행에 대해 죽일 확률 적용
print(drop(x))

tensor([[-0.0000,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
        [-1.9237,  0.0000,  5.3354,  0.0000, -0.0000, -0.0000,  0.0000],
        [ 6.8558,  2.2090,  0.8233,  0.0000,  0.0000,  0.0000,  0.0000]])


In [25]:
class sample_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_layer=nn.Sequential(nn.Linear(3,5),
                                      nn.Dropout(p=0.99), # linear에 나온 값을 죽이는 것
                                      nn.Linear(5,2)) # 그리고 input으로 들어감

    def forward(self, x):
      x = nn.Linear(3,5)(x)
      print(f"linear 이후 x 값" + "-"*20)
      print(x)
      x = nn.Dropout(p=0.99)(x)
      print(f"drop out 적용 값" + "-"*20)
      print(x)
      x = nn.Linear(5,2)(x)
      return x

model=sample_model()
model.train() # train mode로 전환 > dropout은 train & test mode가 다르므로
x=torch.randn(3,3) # 그 담에 x=torch.randn(2,3,5)
model(x)
print("-"*20)
print(model.drop_layer[0].weight)
print(model.drop_layer[2].weight)

linear 이후 x 값--------------------
tensor([[-0.3288, -0.8794,  0.6692, -0.8254,  0.4615],
        [-1.1957,  0.0438,  0.2685, -0.9766,  1.5506],
        [-1.3523, -0.0260,  0.1759, -1.6090,  1.7047]],
       grad_fn=<AddmmBackward0>)
drop out 적용 값--------------------
tensor([[-0., -0., 0., -0., 0.],
        [-0., 0., 0., -0., 0.],
        [-0., -0., 0., -0., 0.]], grad_fn=<MulBackward0>)
--------------------
Parameter containing:
tensor([[-0.2547, -0.5652, -0.5131],
        [ 0.4522, -0.1448, -0.2477],
        [-0.2195,  0.1030, -0.5102],
        [-0.0799,  0.2668, -0.1041],
        [-0.4966, -0.0661, -0.1344]], requires_grad=True)
Parameter containing:
tensor([[-0.3041,  0.3592,  0.2491, -0.2553,  0.3866],
        [-0.2376,  0.1693,  0.1904, -0.3102, -0.1422]], requires_grad=True)


In [27]:
layer = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1) # stride=1, padding=0 이 디폴트
# 2*1*3*3 > 32*2*3*3
layer(torch.randn(32,1,5,5)).shape

torch.Size([32, 2, 5, 5])

In [28]:
layer = nn.Conv2d(3,5,3, stride=2, padding=1) # 5*3*3*3 > 32*5*3*3
print(layer(torch.randn(32,3,5,5)).shape)
print(layer.weight.shape) # filter weight

torch.Size([32, 5, 3, 3])
torch.Size([5, 3, 3, 3])


In [29]:
conv1 = nn.Conv2d(1,8,6,stride=2) # 8*1*6*6 > 8*
x=torch.randn(32,1,28,28)
print(conv1(x).shape)

conv2 = nn.Conv2d(8,16,3,padding=1)
print(conv2(conv1(x)).shape)

torch.Size([32, 8, 12, 12])
torch.Size([32, 16, 12, 12])


In [43]:
maxpool = nn.MaxPool2d(2) # = nn.MaxPool2d(kernel_size=2, stride=(2,2))
x=torch.randn(1,1,6,6)
print(x)
print(maxpool(x))
print(maxpool(torch.randn(32,3,8,8)).shape)
print("-"*20)
print(x[0][0][:2, :2].flatten().max())

tensor([[[[ 0.0527, -1.5883, -1.2564, -1.5420,  1.3056,  1.2560],
          [ 0.2152,  0.5595, -1.0936,  1.8552, -1.7689, -0.1954],
          [-1.7554,  0.6607, -0.5749,  0.6414, -0.0558,  0.7648],
          [-1.2029,  0.1541,  0.6868,  0.6187,  0.2007, -0.5108],
          [ 0.6895, -1.3300, -0.0221, -0.1561,  0.0688, -0.7014],
          [ 2.5362, -1.4334, -1.4457,  0.3805,  0.3336,  1.6405]]]])
tensor([[[[0.5595, 1.8552, 1.3056],
          [0.6607, 0.6868, 0.7648],
          [2.5362, 0.3805, 1.6405]]]])
torch.Size([32, 3, 4, 4])
--------------------
tensor(0.5595)


In [42]:
avgpool=nn.AvgPool2d(2) # = nn.AvgPool2d(kernel_size=2, stride=(2,2))
x=torch.randn(1,1,6,6)
print(x)
print(avgpool(x))
print(avgpool(torch.randn(32,3,6,6)).shape)
print("-"*20)
print(x[0][0][:2, :2].flatten().mean())

tensor([[[[ 0.2454,  0.7802,  0.3128,  2.2294,  0.2362,  1.6833],
          [ 1.2310, -0.0353,  0.7828,  0.7158, -0.2381,  1.0304],
          [-0.7964, -0.9748,  1.4393,  0.0348, -0.4931,  0.7751],
          [-1.8962, -0.1057, -1.4068,  0.0835, -0.2721, -0.5153],
          [-2.2407, -1.3391, -1.0993, -0.6526, -1.0573,  0.7939],
          [ 1.2758,  1.2034,  1.2023,  2.2228,  1.3024, -0.4540]]]])
tensor([[[[ 0.5553,  1.0102,  0.6780],
          [-0.9433,  0.0377, -0.1264],
          [-0.2751,  0.4183,  0.1463]]]])
torch.Size([32, 3, 3, 3])
--------------------
tensor(0.5553)


# Model

In [44]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1,8,6,stride=2)
        self.conv2 = nn.Conv2d(8,16,3,padding=1)
        self.Maxpool2 = nn.MaxPool2d(2) # 32*16*6*6 > flatten 이전까지 그냥 해보고 shape 확인해봐라
        self.fc = nn.Linear(16*6*6,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Maxpool2(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x